In [ ]:
import sqlite3
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Dense

# Connect to the database
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

# Execute SQL query to retrieve data
cursor.execute("SELECT motion, imagesign, tap, phonetic_complexity_data, phonetic_probability_data, iconizity, cluster_labels, gestureclass FROM UserData;")
data = cursor.fetchall()

# Close the database connection
conn.close()

# Check if the fetched data is not empty
if data:
    # Extract individual columns from the fetched data
    motion, imagesign, tap, phonetic_complexity_data, phonetic_probability_data, iconizity_data, cluster_labels, gestureclass = zip(*data)

    # Ensure that the data are concatenated correctly for clustering
    X = np.concatenate((motion, imagesign, tap, phonetic_complexity_data, phonetic_probability_data, iconizity_data, cluster_labels), axis=1)

    # Standardize the numerical features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Set X and y for further processing
    X = X_scaled
    y = np.array(gestureclass)  # Convert to numpy array

    # Split the data into training, validation, and testing sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Define your Keras model
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')  # Use sigmoid activation for binary classification
    ])

    # Compile your model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',  # Use binary cross-entropy for binary classification
                  metrics=['accuracy'])  # Use accuracy as a metric

    # Train your model
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

    # Evaluate your model
    loss, accuracy = model.evaluate(X_test, y_test)

    # Make predictions for the entire dataset
    predictions = model.predict_classes(X)

    # Connect to the database again to update the predictions
    conn = sqlite3.connect('your_database.db')
    cursor = conn.cursor()

    # Update the database with the predictions
    for i, prediction in enumerate(predictions):
        cursor.execute("UPDATE UserData SET gestureclass = ? WHERE rowid = ?", (prediction[0], i+1))

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

else:
    pass  # Handle the case when there's no data fetched